In [37]:
import pandas as pd
import numpy as np
import json

In [38]:
df = pd.read_excel('Data/db_master.xlsx',sheet_name='Dist_Detail_Master')

In [39]:
colu = ['STATE NAME', 'ASM Name', 'SS Code', 'SS NAME', 'SS TOWN',
       'Dist. Code', 'Dealer Name (As Per GST)','Total Target']

In [40]:
df = df[colu]

In [41]:
df.shape

(2140, 8)

In [42]:
df.drop_duplicates(inplace=True)

In [43]:
df['STATE NAME'].unique()

array(['Madhya Pradesh', 'Maharashtra', 'Himachal Pradesh', 'Punjab',
       'Gujarat', 'Uttarakhand', 'Haryana', 'Rajasthan'], dtype=object)

In [44]:
#df = df[df['STATE NAME'].isin(['Maharashtra','Gujarat', 'Rajasthan'])]
df = df.apply(lambda col: col.str.strip().replace(r'\s+', ' ', regex=True) if col.dtype == "object" else col)

In [45]:
df['STATE NAME'] = df['STATE NAME'].str.title()
df['ASM Name'] = df['ASM Name'].str.title()
df['SS NAME'] = df['SS NAME'].str.title()
df['Dealer Name (As Per GST)'] = df['Dealer Name (As Per GST)'].str.title()

In [46]:
df['Total Target'] = df['Total Target'].astype(int,errors='ignore')
df['Total Target'] = df['Total Target'].fillna(0)
df['Total Target'] = df['Total Target'].astype(int,errors='ignore')
df['Dist. Code'] = df['Dist. Code'].fillna('NA')

In [47]:
df['SS'] = df['SS NAME']+"|"+df['SS Code']
df['DB'] = df['Dealer Name (As Per GST)']+"|"+df['Dist. Code']

In [48]:
df.rename(columns={'Total Target':'TGT','STATE NAME':'STATE','ASM Name':'ASM'},inplace=True)

In [49]:
df.columns

Index(['STATE', 'ASM', 'SS Code', 'SS NAME', 'SS TOWN', 'Dist. Code',
       'Dealer Name (As Per GST)', 'TGT', 'SS', 'DB'],
      dtype='object')

In [50]:
df.isna().sum()


STATE                       0
ASM                         0
SS Code                     0
SS NAME                     0
SS TOWN                     0
Dist. Code                  0
Dealer Name (As Per GST)    0
TGT                         0
SS                          0
DB                          0
dtype: int64

In [51]:
col2 = ['STATE','SS','ASM','DB','TGT']

In [52]:
# Google Sheet Fech data
sheet_url = "https://docs.google.com/spreadsheets/d/1Cpcj2FgA0XG8M9Rji_-DNW5LGyYmkrxEmrJ-ee58VMY/export?format=csv"
sheet = pd.read_csv(sheet_url)

In [53]:
sheet.head(5)

,Timestamp,State,SS Name,DB Name,Month,DataType,Category,Stock Id,Stock DB,Stock TGT,Week 1,Week 2,Week 3,Week 4
0,8/3/2025 11:48:00,Rajasthan,Karni Traders|ACUSDK0094,Ashish Bissa,2025-08,TGT,Bis.,Ashish Bissa,Agarwal General Store|DLR00000000174206,315,105.0,0.0,105.0,105.0
1,8/3/2025 11:48:00,Rajasthan,Karni Traders|ACUSDK0094,Ashish Bissa,2025-08,TGT,Bis.,Ashish Bissa,Agarwal Sweet|DLR00000000173474,5410,300.0,1150.0,2000.0,1960.0
2,8/3/2025 11:48:00,Rajasthan,Karni Traders|ACUSDK0094,Ashish Bissa,2025-08,TGT,Bis.,Ashish Bissa,Bajrang Agency - Napasar|DLR00000000169253,380,115.0,100.0,165.0,0.0
3,8/3/2025 11:48:00,Rajasthan,Karni Traders|ACUSDK0094,Ashish Bissa,2025-08,TGT,Bis.,Ashish Bissa,Gupta Agency - Lunkaransar|DLR00000000169270,430,170.0,120.0,0.0,140.0
4,8/3/2025 11:48:00,Rajasthan,Karni Traders|ACUSDK0094,Ashish Bissa,2025-08,TGT,Bis.,Ashish Bissa,Kripa Bherunath -|DLR00000000169281,55,25.0,0.0,30.0,0.0


In [54]:
sheet.columns

Index(['Timestamp', 'State', 'SS Name', 'DB Name', 'Month', 'DataType',
       'Category', 'Stock Id', 'Stock DB', 'Stock TGT', 'Week 1', 'Week 2',
       'Week 3', 'Week 4'],
      dtype='object')

In [55]:
sheet['dbid'] = sheet['Stock DB'].str.split('|').str[-1]


In [56]:
sheet.columns

Index(['Timestamp', 'State', 'SS Name', 'DB Name', 'Month', 'DataType',
       'Category', 'Stock Id', 'Stock DB', 'Stock TGT', 'Week 1', 'Week 2',
       'Week 3', 'Week 4', 'dbid'],
      dtype='object')

In [57]:
sheet1 = sheet[['DataType',
       'Category', 'Stock Id', 'Stock DB', 'Stock TGT', 'Week 1', 'Week 2',
       'Week 3', 'Week 4', 'dbid']]

In [58]:
df1 = df[col2]
df2 = df1
df2['dbid'] = df2['DB'].str.split('|').str[-1]


C:\Users\pc\AppData\Local\Temp\ipykernel_5368\3495708421.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['dbid'] = df2['DB'].str.split('|').str[-1]


In [59]:
merge = pd.merge(df2, sheet1,how='left',left_on='DB',right_on='Stock DB')

In [60]:
merge.columns

Index(['STATE', 'SS', 'ASM', 'DB', 'TGT', 'dbid_x', 'DataType', 'Category',
       'Stock Id', 'Stock DB', 'Stock TGT', 'Week 1', 'Week 2', 'Week 3',
       'Week 4', 'dbid_y'],
      dtype='object')

In [61]:
pvt = pd.pivot_table(data=merge,index=['STATE', 'SS', 'ASM', 'DB','dbid_x'],columns=['DataType','Category'],aggfunc={
    'Week 1':'sum',
    'Week 2':'sum',
    'Week 3':'sum',
    'Week 4':'sum',
})
pvt

Week 1  \
DataType                                                                                                         ACH   
Category                                                                                                        Bis.   
STATE     SS                       ASM            DB                                        dbid_x                     
Gujarat   Ashok Traders|ACUSDA0117 Toshif Mansuri Alpha Sales|DLR00000000170375             DLR00000000170375    NaN   
                                                  Apsara Sales Agency|DLR00000000170429     DLR00000000170429    NaN   
                                                  Ashok Traders|DLR00000000170370           DLR00000000170370    NaN   
                                                  Babaramdev Sales|DLR00000000172315        DLR00000000172315    NaN   
                                                  Bhagwati Tredars|DLR00000000173281        DLR00000000173281    NaN   
...                                                                                                              ...   
Rajasthan Vinod Agency|ACUSDV0142  Satish Singh   Jain Departmental Store|DLR00000000160494 DLR00000000160494    NaN   
                                                  Love Agencies|DLR00000000160496           DLR00000000160496    NaN   
                                                  Rajesh & Brothers|DLR00000000160498       DLR00000000160498    NaN   
                                                  Ramkishor Rajesh Kumar|DLR00000000160495  DLR00000000160495    NaN   
                                                  Vinod Agencies|DLR00000000160506          DLR00000000160506    NaN   

                                                                                                                     \
DataType                                                                                                              
Category                                                                                                      Conf.   
STATE     SS                       ASM            DB                                        dbid_x                    
Gujarat   Ashok Traders|ACUSDA0117 Toshif Mansuri Alpha Sales|DLR00000000170375             DLR00000000170375   NaN   
                                                  Apsara Sales Agency|DLR00000000170429     DLR00000000170429   NaN   
                                                  Ashok Traders|DLR00000000170370           DLR00000000170370   NaN   
                                                  Babaramdev Sales|DLR00000000172315        DLR00000000172315   NaN   
                                                  Bhagwati Tredars|DLR00000000173281        DLR00000000173281   NaN   
...                                                                                                             ...   
Rajasthan Vinod Agency|ACUSDV0142  Satish Singh   Jain Departmental Store|DLR00000000160494 DLR00000000160494   NaN   
                                                  Love Agencies|DLR00000000160496           DLR00000000160496   NaN   
                                                  Rajesh & Brothers|DLR00000000160498       DLR00000000160498   NaN   
                                                  Ramkishor Rajesh Kumar|DLR00000000160495  DLR00000000160495   NaN   
                                                  Vinod Agencies|DLR00000000160506          DLR00000000160506   NaN   

                                                                                                                       \
DataType                                                                                                                
Category                                                                                                      Noodles   
STATE     SS                       ASM            DB                                        dbid_x                      
Gujarat   Ashok Traders|ACUSDA0117 Tosh

In [62]:
cols = pvt.columns

# Sort the columns as per your custom rule: TGT before ACH
sorted_cols = sorted(cols, key=lambda x: (x[0], 0 if x[1] == 'TGT' else 1, x[2]))

# Reassign sorted columns to DataFrame
pvt1 = pvt[sorted_cols]


In [63]:
pvt1.reset_index().to_excel('Data/Checklist.xlsx')

In [64]:
df1['ASM'].unique()

array(['Ankit Bhardwaj', 'Brajesh Sharma', 'Vacant', 'Shailesh Surve',
       'Parmod Kumar', 'Anand Gupta', 'Neeraj Kapoor', 'Girvat Raj',
       'Avneesh', 'Rakesh Kumar', 'Badal Srivastava', 'Toshif Mansuri',
       'Jasbir Verma', 'Vinay Pandey', '', 'Kapil Sharma',
       'Vinesh Trivedi', 'Raj Kumar', 'Prakash Lohani', 'Shymu Verma',
       'Salabh Partap', 'Ajit Pal', 'Sushil Kumar', 'Shashi Mishra',
       'Parvesh Sharma', 'Premdeep Deshwal', 'Mahesh',
       'Anil Jamadar Singh', 'Hitesh Thakker', 'Rajinder Singh',
       'Jasbeer Verma', 'Rajiv Kumar', 'Sahil Mansuri', 'Dinesh Sharma',
       'Amit Patil', 'Puneet Kansara', 'Ajay Sachdeva', 'Lavish Sethi',
       'Cp Gaur', 'Satish Singh', 'Mahendra Pal Singh', 'Sarwan Sharma',
       'Ashish Bissa', 'Subhash Yogi', 'Yogesh Sharma'], dtype=object)

# Read ASM Password

In [65]:
pas = pd.read_excel('Data/asm.xlsx',usecols=['ASM','Password'])

In [66]:
df2 = df1[['STATE','SS','ASM']].drop_duplicates()
mer = pd.merge(df2,pas,on='ASM',how='left')
mer['Password'] = mer['Password'].fillna(2025)
df2 = mer

In [67]:
df3 = df1[['SS','ASM','DB','TGT']]

In [68]:
df3.sort_values(by=['DB','ASM']).to_json('Output/dbtgt.json')

In [69]:
df3.columns

Index(['SS', 'ASM', 'DB', 'TGT'], dtype='object')

# SS to ASM

In [70]:
# Create nested dictionary with password at DB level
nested = {}

for _, row in df2.iterrows():
    state = row['STATE']
    ss = row['SS']
    db = row['ASM'] # DB Name
    pwd = row['Password']

    state_dict = nested.setdefault(state, {})
    ss_dict = state_dict.setdefault(ss, {})
    db_dict = ss_dict.setdefault(db, {"password": pwd})


# Save
with open("Output/db.json", "w") as f:
    json.dump(nested, f, indent=4)

In [71]:
df2.to_excel('password.xlsx')